In [ ]:
import cv2
import math
import numpy as np

image_path =  "C:\\Users\\rahul\\OneDrive\\Desktop\\Python\\fish.jpg"
img1=cv2.imread(image_path)

# Fisheye correction parameters
cam_heading = 90.0
cam_pitch = 90.0
cam_fov = 90.0
src_width, src_height =img1.shape[1],img1.shape[0]
dest_width = 720
dest_height = 480

# Initialize camera parameters
DEG2RAD = math.pi / 180.0

# Calculate camera parameters
theta_fac = src_height / math.pi
phi_fac = src_width * 0.5 / math.pi
ratioUp = 2.0 * math.tan(cam_fov * DEG2RAD / 2.0)
ratioRight = ratioUp * 1.33

camDirX = math.sin(cam_pitch * DEG2RAD) * math.sin(cam_heading * DEG2RAD)
camDirY = math.cos(cam_pitch * DEG2RAD)
camDirZ = math.sin(cam_pitch * DEG2RAD) * math.cos(cam_heading * DEG2RAD)

camUpX = ratioUp * math.sin((cam_pitch - 90.0) * DEG2RAD) * math.sin(cam_heading * DEG2RAD)
camUpY = ratioUp * math.cos((cam_pitch - 90.0) * DEG2RAD)
camUpZ = ratioUp * math.sin((cam_pitch - 90.0) * DEG2RAD) * math.cos(cam_heading * DEG2RAD)

camRightX = ratioRight * math.sin((cam_heading - 90.0) * DEG2RAD)
camRightY = 0.0
camRightZ = ratioRight * math.cos((cam_heading - 90.0) * DEG2RAD)

camPlaneOriginX = camDirX + 0.5 * camUpX - 0.5 * camRightX
camPlaneOriginY = camDirY + 0.5 * camUpY - 0.5 * camRightY
camPlaneOriginZ = camDirZ + 0.5 * camUpZ - 0.5 * camRightZ
FOV = math.pi * 130.0 / 180.0  # FOV of the fisheye, e.g., 180 degrees

def correct_fisheye(input_image_path):
    img = cv2.imread(input_image_path)
    cv2.imshow("initial",img)
    src_width = img.shape[1]
    src_height = img.shape[0]
    
    # Calculate camera parameters
    theta_fac = src_height / math.pi
    phi_fac = src_width * 0.5 / math.pi
    ratioUp = 2.0 * math.tan(cam_fov * DEG2RAD / 2.0)
    ratioRight = ratioUp * 1.33

    camDirX = math.sin(cam_pitch * DEG2RAD) * math.sin(cam_heading * DEG2RAD)
    camDirY = math.cos(cam_pitch * DEG2RAD)
    camDirZ = math.sin(cam_pitch * DEG2RAD) * math.cos(cam_heading * DEG2RAD)

    camUpX = ratioUp * math.sin((cam_pitch - 90.0) * DEG2RAD) * math.sin(cam_heading * DEG2RAD)
    camUpY = ratioUp * math.cos((cam_pitch - 90.0) * DEG2RAD)
    camUpZ = ratioUp * math.sin((cam_pitch - 90.0) * DEG2RAD) * math.cos(cam_heading * DEG2RAD)

    camRightX = ratioRight * math.sin((cam_heading - 90.0) * DEG2RAD)
    camRightY = 0.0
    camRightZ = ratioRight * math.cos((cam_heading - 90.0) * DEG2RAD)

    camPlaneOriginX = camDirX + 0.5 * camUpX - 0.5 * camRightX
    camPlaneOriginY = camDirY + 0.5 * camUpY - 0.5 * camRightY
    camPlaneOriginZ = camDirZ + 0.5 * camUpZ - 0.5 * camRightZ
    FOV = math.pi * 130.0 / 180.0  # FOV of the fisheye, e.g., 180 degrees

    size = dest_height, dest_width, 3
    dest = np.zeros(size, dtype=float)

    for i in range(1, dest_height):
        for j in range(1, dest_width):
            fx = float(j) / float(dest_width)
            fy = float(i) / float(dest_height)
            rayY = camPlaneOriginX + fx * camRightX - fy * camUpX
            rayX = camPlaneOriginY + fx * camRightY - fy * camUpY
            rayZ = camPlaneOriginZ + fx * camRightZ - fy * camUpZ
            rayNorm = math.sqrt(rayX * rayX + rayZ * rayZ)

            # Calculate fisheye angle and radius
            theta = math.atan2(rayZ, rayX)
            phi = math.atan2(rayNorm, rayY)
            r = src_height * phi / FOV

            # Pixel in fisheye space
            theta_i = math.floor(0.5 * src_width + r * math.sin(theta))
            phi_i = math.floor(0.5 * src_height - r * math.cos(theta))

            dest_offset = (i * dest_width + j)
            src_offset = (phi_i * src_width + theta_i)
            y = max(min(int(phi_i), src_height - 1), 1)
            x = max(min(int(theta_i), src_width - 1), 1)

            # Implement interpolation
            dest[i, j, 0] = img[y, x, 0] / 255.0
            dest[i, j, 1] = img[y, x, 1] / 255.0
            dest[i, j, 2] = img[y, x, 2] / 255.0

    return dest * 255

#Returns edges detected in an image
def canny_edge_detector(frame):
    
    # Convert to grayscale as only image intensity needed for gradients
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # 5x5 gaussian blur to reduce noise 
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    cv2.imshow("blurred image",blur)
    
    # Canny edge detector with minVal of 50 and maxVal of 150
    canny = cv2.Canny(blur, 50, 150)
    cv2.imshow("canny image",canny)
    
    return canny  

# Returns a masked image
def ROI_mask(image):
    
    height = image.shape[0]
    width = image.shape[1]

    
    # A triangular polygon to segment the lane area and discarded other irrelevant parts in the image
    # Defined by three (x, y) coordinates    
    polygons = np.array([ 
        [(0, height), (round(width/2), round(height/2)), (1000, height)] 
        ]) 
    
    mask = np.zeros_like(image) 
    cv2.fillPoly(mask, polygons, 255)  ## 255 is the mask color
    
    # Bitwise AND between canny image and mask image
    masked_image = cv2.bitwise_and(image, mask)
    cv2.imshow("masked image",masked_image)
    
    return masked_image

def get_coordinates (image, params):
     
    slope, intercept = params 
    y1 = image.shape[0]     
    y2 = int(y1 * (3/5)) # Setting y2 at 3/5th from y1
    x1 = int((y1 - intercept) / slope) # Deriving from y = mx + c
    x2 = int((y2 - intercept) / slope) 
    
    return np.array([x1, y1, x2, y2])
# Returns averaged lines on left and right sides of the image
def avg_lines(image, lines): 
    
    left = [] 
    right = [] 
    
    for line in lines: 
        x1, y1, x2, y2 = line.reshape(4)
          
        # Fit polynomial, find intercept and slope 
        params = np.polyfit((x1, x2), (y1, y2), 1)  
        slope = params[0] 
        y_intercept = params[1] 
        
        if slope < 0: 
            left.append((slope, y_intercept)) #Negative slope = left lane
        else: 
            right.append((slope, y_intercept)) #Positive slope = right lane
    
    # Avg over all values for a single slope and y-intercept value for each line
    
    left_avg = np.average(left, axis = 0) 
    right_avg = np.average(right, axis = 0) 
    
    # Find x1, y1, x2, y2 coordinates for left & right lines
    left_line = get_coordinates(image, left_avg) 
    right_line = get_coordinates(image, right_avg)
    
    return np.array([left_line, right_line])

# Draws lines of given thickness over an image
def draw_lines(image, lines, thickness): 
   
    print(lines)
    line_image = np.zeros_like(image)
    color=[0, 0, 255]
    
    if lines is not None: 
        for x1, y1, x2, y2 in lines:
                    cv2.line(line_image, (x1, y1), (x2, y2), color, thickness)

            
    # Merge the image with drawn lines onto the original.
    combined_image = cv2.addWeighted(image, 0.8, line_image, 1.0, 0.0)
    cv2.imshow("combined image",combined_image)
    
    return combined_image

# Load input image
image_path =  "C:\\Users\\rahul\\OneDrive\\Desktop\\Python\\fish.jpg"
corrected_image = correct_fisheye(image_path)
corrected_image = corrected_image.astype(np.uint8)
cv2.imshow("corrected fisheyee image",corrected_image)


# Lane detection
canny_edges = canny_edge_detector(corrected_image)
cropped_image = ROI_mask(canny_edges)
lines = cv2.HoughLinesP(
    cropped_image,
    rho=2,
    theta=np.pi / 180,
    threshold=100,
    lines=np.array([]),
    minLineLength=40,
    maxLineGap=25
)
averaged_lines = avg_lines(corrected_image, lines)
lane_detected_image = draw_lines(corrected_image, averaged_lines, 5)

# Display the corrected image with lane detection
cv2.imshow("Lane Detection", lane_detected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[[ 38 480 347 288]
 [608 480 330 288]]
